In [1]:
import pandas as pd
pd.options.mode.chained_assignment = None
import numpy as np

import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

In [2]:
data_pricing =  pd.read_csv("get_around_pricing_project.csv")
data_delay = pd.read_excel("get_around_delay_analysis.xlsx")

In [3]:
display (data_pricing.head(2))
display (data_pricing.shape)
display (data_delay.head(2))
display (data_delay.shape)

,Unnamed: 0,model_key,mileage,engine_power,fuel,paint_color,car_type,private_parking_available,has_gps,has_air_conditioning,automatic_car,has_getaround_connect,has_speed_regulator,winter_tires,rental_price_per_day
0,0,Citroën,140411,100,diesel,black,convertible,True,True,False,False,True,True,True,106
1,1,Citroën,13929,317,petrol,grey,convertible,True,True,False,False,False,True,True,264


(4843, 15)

,rental_id,car_id,checkin_type,state,delay_at_checkout_in_minutes,previous_ended_rental_id,time_delta_with_previous_rental_in_minutes
0,505000.0,363965.0,mobile,canceled,NaN,NaN,NaN
1,507750.0,269550.0,mobile,ended,-81.0,NaN,NaN


(21310, 7)

In [4]:
data_delay.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21310 entries, 0 to 21309
Data columns (total 7 columns):
 #   Column                                      Non-Null Count  Dtype  
---  ------                                      --------------  -----  
 0   rental_id                                   21310 non-null  float64
 1   car_id                                      21310 non-null  float64
 2   checkin_type                                21310 non-null  object 
 3   state                                       21310 non-null  object 
 4   delay_at_checkout_in_minutes                16346 non-null  float64
 5   previous_ended_rental_id                    1841 non-null   float64
 6   time_delta_with_previous_rental_in_minutes  1841 non-null   float64
dtypes: float64(5), object(2)
memory usage: 1.1+ MB


- Lot of missing values in the column "previous_ended_rental_id" this will partially prevent us from finding the real impact of lateness (mostly if it leads to cancelation or not)

##### Conclusion

Missing value on "previous_ended_rental_id" is an issue because we need it to determine if the previous rental did a late checkout in order to try to find how lateness impact the business.

In [5]:
# Findind number of unique cars
number_of_cars = len(data_delay['car_id'].unique())

# Finding average rental price
car_average_rental_price_per_day = round(data_pricing["rental_price_per_day"].mean())

# Finding number of rentals
number_of_rentals = data_delay.shape[0]

print (f"There are {number_of_cars} different cars in the dataset 'data_delay'")
print (f"The average rental price of a car per day is {car_average_rental_price_per_day}$ in the dataset 'data_pricing'")
print (f"There are {number_of_rentals} rentals in the dataset 'data_delay'")

There are 8143 different cars in the dataset 'data_delay'
The average rental price of a car per day is 121$ in the dataset 'data_pricing'
There are 21310 rentals in the dataset 'data_delay'


In [6]:
# Removing NaN values about late checkout from the dataset
data_delay_without_nan = data_delay[data_delay["delay_at_checkout_in_minutes"].isna() == False]

print (f"There are {data_delay_without_nan.shape[0]} rentals in the dataset 'data_delay_without_nan'")

def check_if_late(late):
    if late > 0:
        return "late"
    else:
        return "not late"

# Creating a new column to categorize lateness
data_delay_without_nan["delay"] = data_delay_without_nan["delay_at_checkout_in_minutes"].apply(check_if_late)

# Creating a new DataFrame with values counts for the lateness category
new_df = (data_delay_without_nan['delay'].value_counts(normalize=True)*100).rename_axis('delay').reset_index(name='counts')

# Bar chart
fig = px.bar(new_df, 
            x="delay", 
            y="counts", 
            color="delay", 
            title="----------", 
            color_discrete_sequence=['royalblue', '#880808'],
            text="counts",
            width=1000, height=500,)
fig.update_traces(texttemplate='%{text:.2f}' + "%", textposition='outside')
fig.update_layout(title_x=0.5, yaxis={'visible': False}, xaxis={'visible': True}, legend_title="", title_text="Proportion of lateness - Whole Dataset after removal of NaN", template='plotly_dark')
fig.show()

There are 16346 rentals in the dataset 'data_delay_without_nan'


In [7]:
# Creating a new DataFrame with values counts for the state category
df_rental_state = (data_delay['state'].value_counts(normalize=True)*100).rename_axis('state').reset_index(name='counts')

# Bar chart
fig = px.bar(df_rental_state, 
            x="state", 
            y="counts", 
            color="state", 
            title="----------", 
            color_discrete_sequence=['#03c04A', '#80471C'],
            text="counts",
            width=1000, height=500,)
fig.update_traces(texttemplate='%{text:.2f}' + "%", textposition='outside')
fig.update_layout(title_x=0.5, yaxis={'visible': False}, xaxis={'visible': True}, legend_title="", title_text="Proportion of rentals per state - Whole Dataset", template='plotly_dark')
fig.show()

In [8]:
# Making a list of all the previous_ended_rental_id
lst_previous_rental_id = data_delay["previous_ended_rental_id"]

# Removing all NaN values from this list
lst_previous_rental_id = [x for x in lst_previous_rental_id if np.isnan(x) == False]

# Creating a new df with all the rental id found in my previous list to see if they were late or not
df_previous_rental_id = data_delay[data_delay["rental_id"].isin(lst_previous_rental_id)]

# Merging the two df to have a the same row both the previous rental id and the following rental id
df_merged = df_previous_rental_id.merge(data_delay, how='inner', left_on='rental_id', right_on='previous_ended_rental_id')

# Removing useless columns
df_merged.drop(['state_x', 'previous_ended_rental_id_y', 'previous_ended_rental_id_x', 'time_delta_with_previous_rental_in_minutes_x', 'car_id_y', 'delay_at_checkout_in_minutes_y'], axis=1, inplace=True)
print (f"After cleaning the dataset, there are {df_merged.shape[0]} rows left to use from {len(data_delay)} rows originally.")

# Creating a new DataFrame when checkout was late
df_merged_and_late = df_merged[df_merged["delay_at_checkout_in_minutes_x"] > 0]

# Creating a new DataFrame when checkout was done on time
df_merged_and_not_late = df_merged[df_merged["delay_at_checkout_in_minutes_x"] <= 0]

# Creating a new DataFrame with values counts of state for late checkout
df_merged_and_late_value_counts = (df_merged_and_late['state_y'].value_counts(normalize=True)*100).rename_axis('state_y').reset_index(name='counts')

# Creating a new DataFrame with values counts of state for checkout on time
df_merged_and_not_late_value_counts = (df_merged_and_not_late['state_y'].value_counts(normalize=True)*100).rename_axis('state_y').reset_index(name='counts')

# Bar chart - state for late checkout
fig = px.bar(df_merged_and_late_value_counts, 
            x="state_y", 
            y="counts", 
            color="state_y", 
            title="----------", 
            color_discrete_sequence=['#03c04A', '#80471C'],
            text="counts",
            width=1000, height=500,)
fig.update_traces(texttemplate='%{text:.2f}' + "%", textposition='outside')
fig.update_layout(title_x=0.5, yaxis={'visible': False}, xaxis={'visible': True}, legend_title="", title_text="Cancelation rate after late checkout", template='plotly_dark')
fig.show()

# Bar chart - state for checkout on time
fig2 = px.bar(df_merged_and_not_late_value_counts, 
            x="state_y", 
            y="counts", 
            color="state_y", 
            title="----------", 
            color_discrete_sequence=['#03c04A', '#80471C'],
            text="counts",
            width=1000, height=500,)
fig2.update_traces(texttemplate='%{text:.2f}' + "%", textposition='outside')
fig2.update_layout(title_x=0.5, yaxis={'visible': False}, xaxis={'visible': True}, legend_title="", title_text="Cancelation rate if checkout was not late", template='plotly_dark')
fig2.show()

After cleaning the dataset, there are 1841 rows left to use from 21310 rows originally.


In [9]:
# Creating a new column to find if the checkout of the previous rental happened after the start of the following rental
df_merged_and_late["wait_time_in_minutes"] = df_merged_and_late["delay_at_checkout_in_minutes_x"] - df_merged_and_late["time_delta_with_previous_rental_in_minutes_y"]

# Keeping only cases when the checkout of the previous rental happened after the start of the following rental
df_merged_way_too_late = df_merged_and_late[df_merged_and_late["wait_time_in_minutes"] > 0]
print (f"We have {df_merged_way_too_late['state_y'].value_counts()[0]} ended rentals when the checkout of the previous rental happened after the start of the following rental")
print (f"We have {df_merged_way_too_late['state_y'].value_counts()[1]} canceled rentals when the checkout of the previous rental happened after the start of the following rental")

# Creating a new DataFrame with values counts of state when the checkout of the previous rental happened after the start of the following rental
df_merged_way_too_late2 = (df_merged_way_too_late['state_y'].value_counts(normalize=True)*100).rename_axis('state_y').reset_index(name='counts')


# Bar chart
fig = px.bar(df_merged_way_too_late2, 
            x="state_y", 
            y="counts", 
            color="state_y", 
            title="----------", 
            color_discrete_sequence=['#03c04A', '#80471C'],
            text="counts",
            width=1000, height=500)
            
fig.update_traces(texttemplate='%{text:.2f}' + "%", textposition='outside')
fig.update_layout(title_x=0.5, yaxis={'visible': False}, xaxis={'visible': True}, legend_title="", title_text="Proportion of state when checkout happened after the expected start of the following rental", template='plotly_dark', xaxis_title='', showlegend=False)
fig.show()

We have 181 ended rentals when the checkout of the previous rental happened after the start of the following rental
We have 37 canceled rentals when the checkout of the previous rental happened after the start of the following rental


In [10]:
# Checking if there is a difference in cancelation rate between mobile and connect checking type
df_merged_and_late_mobile = df_merged_and_late[df_merged_and_late["checkin_type_x"] == "mobile"]

df_merged_and_late_connect = df_merged_and_late[df_merged_and_late["checkin_type_x"] == "connect"]

print (f"Connect checkin has a {round((df_merged_and_late_connect['state_y'].value_counts(normalize=True)*100)[1],2)}% cancelation rate")
print (f"Mobile checkin has a {round((df_merged_and_late_mobile['state_y'].value_counts(normalize=True)*100)[1],2)}% cancelation rate")

Connect checkin has a 18.73% cancelation rate
Mobile checkin has a 8.71% cancelation rate


##### Conclusion from visualisations and data exploration

- 21310 rentals
- 57,53% have a late checkout after removing NaN from this column (16346 rentals left)
- 1841 rentals left usable after merging lateness on previous rental with its following rental
- 12,14% cancelation if previous checkout was late
- 11,68% cancelation if previous checkout was not late
- 16,97% cancelation if previous checkout was so late it happened after the start of the next rental
- In case of lateness, we have 18.73% cancelation rate for Connect checkin versus 8.71% cancelation rate for Mobile checkin

Lateness have an impact on the cancelation rate but it is lower that what I would have expected before checking the value. It's safe to assume this is not the main cause for cancelation. Given the amount of missing values, I am going to use proportion to find how much rentals would have avoided cancelation and how much rentals would not have been taken if we had set a minimum delta time between rentals. I am also going to check specifically the difference it would make if the feature was applied only on Connect checkin type.

In [11]:
df_merged

,rental_id_x,car_id_x,checkin_type_x,delay_at_checkout_in_minutes_x,rental_id_y,checkin_type_y,state_y,time_delta_with_previous_rental_in_minutes_y
0,531158.0,404169.0,mobile,29.0,534827.0,mobile,ended,90.0
1,533303.0,336902.0,mobile,-340.0,552606.0,mobile,ended,600.0
2,533380.0,314456.0,connect,-167.0,542971.0,connect,ended,690.0
3,534820.0,373893.0,connect,-576.0,543571.0,connect,ended,150.0
4,535313.0,396552.0,mobile,23.0,513104.0,mobile,ended,720.0
...,...,...,...,...,...,...,...,...
1836,574571.0,341861.0,connect,-54.0,571038.0,connect,ended,540.0
1837,574596.0,356190.0,mobile,10.0,575497.0,mobile,ended,30.0
1838,567694.0,349618.0,mobile,-17.0,572870.0,mobile,ended,210.0
1839,568465.0,390871.0,connect,NaN,566228.0,connect,canceled,60.0


In [12]:
# Chose your check-in type
checkin_type = input("Chose the check-in type. Type 'mobile', 'connect' or 'both': ")

while checkin_type != "mobile" and checkin_type != "connect" and checkin_type != "both":
    checkin_type = input("You have to chose between 'mobile', 'connect' or 'both': ")

# Creating a new DataFrame with only ended state
df_true_good_cases = data_delay[data_delay["state"] == "ended"]

# Creating a new DataFrame with all the cases having the data I need
df_good_cases = df_merged.copy()
df_good_cases = df_good_cases[df_good_cases["state_y"] == "ended"]

# If checkin_type is not both, I filter the two previous DataFrames
if checkin_type != "both":
    df_good_cases = df_good_cases[df_good_cases["checkin_type_x"] == checkin_type]
    df_true_good_cases = df_true_good_cases[df_true_good_cases["checkin_type"] == checkin_type]

# Asking new time delta
new_delta = int(input("Please enter the new delta time in minutes: "))

# I keep only the rows where the time delta is higher or equal than the new time delta
df_new_time_delta_good_cases = df_good_cases[df_good_cases["time_delta_with_previous_rental_in_minutes_y"] >= new_delta]

# Creating a money lost variable
money_lost = round(round((len(df_good_cases) - len(df_new_time_delta_good_cases))/len(df_good_cases)*100, 2) * len(df_true_good_cases) / 100) * car_average_rental_price_per_day

print (f"With your new minimum time delta of {new_delta} minutes, {len(df_new_time_delta_good_cases)} cases are good. We lost {len(df_good_cases) - len(df_new_time_delta_good_cases)} cases from our whole {len(df_good_cases)} true good cases.")
print (f"{len(df_good_cases) - len(df_new_time_delta_good_cases)} represents {round((len(df_good_cases) - len(df_new_time_delta_good_cases))/len(df_good_cases)*100, 2)}% of the cases.")
print (f"If it was on the whole dataset of {len(df_true_good_cases)} true good cases. {round((len(df_good_cases) - len(df_new_time_delta_good_cases))/len(df_good_cases)*100, 2)}% would have been a loss of {round(round((len(df_good_cases) - len(df_new_time_delta_good_cases))/len(df_good_cases)*100, 2) * len(df_true_good_cases) / 100)} true good cases for {money_lost}$.")

With your new minimum time delta of 120 minutes, 1020 cases are good. We lost 592 cases from our whole 1612 true good cases.
592 represents 36.72% of the cases.
If it was on the whole dataset of 18045 true good cases. 36.72% would have been a loss of 6626 true good cases for 801746$.


In [13]:
df_merged

,rental_id_x,car_id_x,checkin_type_x,delay_at_checkout_in_minutes_x,rental_id_y,checkin_type_y,state_y,time_delta_with_previous_rental_in_minutes_y
0,531158.0,404169.0,mobile,29.0,534827.0,mobile,ended,90.0
1,533303.0,336902.0,mobile,-340.0,552606.0,mobile,ended,600.0
2,533380.0,314456.0,connect,-167.0,542971.0,connect,ended,690.0
3,534820.0,373893.0,connect,-576.0,543571.0,connect,ended,150.0
4,535313.0,396552.0,mobile,23.0,513104.0,mobile,ended,720.0
...,...,...,...,...,...,...,...,...
1836,574571.0,341861.0,connect,-54.0,571038.0,connect,ended,540.0
1837,574596.0,356190.0,mobile,10.0,575497.0,mobile,ended,30.0
1838,567694.0,349618.0,mobile,-17.0,572870.0,mobile,ended,210.0
1839,568465.0,390871.0,connect,NaN,566228.0,connect,canceled,60.0


In [14]:
# Creating a new DataFrame with only canceled state
df_canceled_cases = df_merged[df_merged["state_y"] == "canceled"]

# If checkin_type is not both, I filter my previous DataFrames
if checkin_type != "both":
    df_canceled_cases = df_canceled_cases[df_canceled_cases["checkin_type_x"] == checkin_type]
    df_merged = df_merged[df_merged["checkin_type_x"] == checkin_type]
    data_delay = data_delay[data_delay["checkin_type"] == checkin_type]

# Finding amount of rows having all the data I need
row_all_data = len(df_merged)
print (f"I have {row_all_data} rows with all the values I need from the {len(data_delay)} rows in the main dataset.")
print (f"{len(df_canceled_cases)} of these {row_all_data} cases were canceled.")

# Creating a new DataFrame with canceled and late cases
df_canceled_and_late_cases = df_canceled_cases[df_canceled_cases["delay_at_checkout_in_minutes_x"] > 0]
print (f"{len(df_canceled_and_late_cases)} cases were canceled and late.")

# Creating a new delay column to find out if the next rental owner had to wait or not
df_canceled_and_late_cases["delay"] = df_canceled_and_late_cases["delay_at_checkout_in_minutes_x"] - df_canceled_and_late_cases["time_delta_with_previous_rental_in_minutes_y"]

# I keep only cases where the next owner had to wait
df_canceled_and_late_cases = df_canceled_and_late_cases[df_canceled_and_late_cases["delay"] > 0]

# I find the percentage of cases where the next owner had to wait
rate_too_late_canceled_case = len(df_canceled_and_late_cases) * 100 / row_all_data
print (f"{len(df_canceled_and_late_cases)} cases were canceled and late and the next owner did not have the car on time. Which represent {round(rate_too_late_canceled_case, 2)}% of the whole cases.")

# Creating a new DataFrame with cases where the next owner had to wait
df_new_time_delta_bad_cases = df_canceled_and_late_cases[df_canceled_and_late_cases["delay"] > new_delta]

# Finding number of cancelation prevented
cancelation_prevented = len(df_canceled_and_late_cases) - len(df_new_time_delta_bad_cases)

# Finding percentage of cancelation prevented
rate_cancelation_prevented =  (cancelation_prevented) * 100 / len(df_canceled_and_late_cases)


print (f"With your new minimum time delta of {new_delta} minutes, {len(df_new_time_delta_bad_cases)} cases are still bad. We prevented {cancelation_prevented} cases from our whole {len(df_canceled_and_late_cases)} true bad cases. Which represents {round(rate_cancelation_prevented,2)}% of the cases.")

total_case_canceled_due_to_lateness = round(len(data_delay) * (rate_too_late_canceled_case/100))

print (f"Proportionally speaking, over {len(data_delay)}, without any minimum time delta. We would have {total_case_canceled_due_to_lateness} cases canceled due to lateness for a total of {total_case_canceled_due_to_lateness * car_average_rental_price_per_day}$.")

cancelation_prevented = round(total_case_canceled_due_to_lateness * (rate_cancelation_prevented/100))

money_saved = cancelation_prevented * car_average_rental_price_per_day

print (f"Proportionally speaking, over {len(data_delay)}, with a minimum time delta of {new_delta} minutes, We would have {cancelation_prevented} cancelation cases prevented for a total of {money_saved}$ saved.")

I have 1841 rows with all the values I need from the 21310 rows in the main dataset.
229 of these 1841 cases were canceled.
106 cases were canceled and late.
37 cases were canceled and late and the next owner did not have the car on time. Which represent 2.01% of the whole cases.
With your new minimum time delta of 120 minutes, 16 cases are still bad. We prevented 21 cases from our whole 37 true bad cases. Which represents 56.76% of the cases.
Proportionally speaking, over 21310, without any minimum time delta. We would have 428 cases canceled due to lateness for a total of 51788$.
Proportionally speaking, over 21310, with a minimum time delta of 120 minutes, We would have 243 cancelation cases prevented for a total of 29403$ saved.


In [15]:
print (f"With {checkin_type} check-in type and a delta of {new_delta} we saved {money_saved}$ but we lost {money_lost}$.")

data = {"Check-in type: " + checkin_type:['Lost', 'Saved'], 
        'Amount':[money_lost, money_saved]}

df = pd.DataFrame(data)
display(df)


# Bar chart
fig = px.bar(df, 
            x="Check-in type: " + checkin_type, 
            y="Amount", 
            color="Check-in type: " + checkin_type, 
            title="----------", 
            color_discrete_sequence=['#FF0000', '#FFD700'],
            text="Amount",
            width=1000, height=500)
            
fig.update_traces(texttemplate='%{text:.0f}' + " $", textposition='outside')
fig.update_layout(title_x=0.5, yaxis={'visible': False}, xaxis={'visible': True}, legend_title="", title_text="Effect on money with a minimum time delta of " + str(new_delta), template='plotly_dark')
fig.show()

With both check-in type and a delta of 120 we saved 29403$ but we lost 801746$.


,Check-in type: both,Amount
0,Lost,801746
1,Saved,29403
